In [24]:
from transformers import pipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [3]:
classification=pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
res=classification(["It is not good that you are good", "I am good but not good", "I am not good but good"])

In [5]:
for result in res:
  print(result)

{'label': 'NEGATIVE', 'score': 0.9996683597564697}
{'label': 'NEGATIVE', 'score': 0.8589476346969604}
{'label': 'POSITIVE', 'score': 0.9982120990753174}


In [6]:
model_name="distilbert/distilbert-base-uncased-finetuned-sst-2-english"

In [16]:
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [17]:
classif=pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [22]:
tokens=tokenizer.tokenize("Hugging Face provides powerful tools and pre-trained models for natural language processing tasks, making it easier to build and deploy AI applications.")
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
input_ids=tokenizer("Hugging Face provides powerful tools and pre-trained models for natural language processing tasks, making it easier to build and deploy AI applications.")

In [23]:
print(tokens)
print(tokens_ids)
print(input_ids)

['hugging', 'face', 'provides', 'powerful', 'tools', 'and', 'pre', '-', 'trained', 'models', 'for', 'natural', 'language', 'processing', 'tasks', ',', 'making', 'it', 'easier', 'to', 'build', 'and', 'deploy', 'ai', 'applications', '.']
[17662, 2227, 3640, 3928, 5906, 1998, 3653, 1011, 4738, 4275, 2005, 3019, 2653, 6364, 8518, 1010, 2437, 2009, 6082, 2000, 3857, 1998, 21296, 9932, 5097, 1012]
{'input_ids': [101, 17662, 2227, 3640, 3928, 5906, 1998, 3653, 1011, 4738, 4275, 2005, 3019, 2653, 6364, 8518, 1010, 2437, 2009, 6082, 2000, 3857, 1998, 21296, 9932, 5097, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [12]:
rest=classif()

In [13]:
for re in rest:
  print(re)

{'label': 'NEGATIVE', 'score': 0.9996683597564697}
{'label': 'POSITIVE', 'score': 0.9996941089630127}
{'label': 'POSITIVE', 'score': 0.9982120990753174}


In [25]:
X_train=["It is not good that you are good", "I am not good but very good", "I am not good but good"]

In [27]:
batch=tokenizer(X_train,padding=True,truncation=True,max_length=512,return_tensors="pt")

In [34]:
with torch.no_grad():
  output=model(**batch)
  print(outputs)
  prediction=F.softmax(outputs.logits,dim=1)
  print(prediction)
  labels=torch.argmax(predictions,dim=1)
  print(labels)
  labels=[model.config.id2label[label_id] for label_id in labels.tolist()]
  print(labels)



SequenceClassifierOutput(loss=None, logits=tensor([[ 4.4311, -3.5801],
        [-3.8883,  4.2037],
        [-3.0326,  3.2923]]), hidden_states=None, attentions=None)
tensor([[9.9967e-01, 3.3162e-04],
        [3.0587e-04, 9.9969e-01],
        [1.7879e-03, 9.9821e-01]])
tensor([0, 1, 1])
['NEGATIVE', 'POSITIVE', 'POSITIVE']


In [36]:
### Saving
save_directory="saved"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)


In [38]:
##Load the model from the train directory

tokenizer=AutoTokenizer.from_pretrained(save_directory)
model=AutoModelForSequenceClassification.from_pretrained(save_directory)